In [44]:
import torch
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import pickle
import pandas as pd
from utilities import CustomDataset, trainLoop, testLoop, save_stats

In [45]:
from typing import List

class SiameseMLPV2(nn.Module):
    def __init__(self, input_size: int, hidden_layers: List[int], output_size: int):
        super(SiameseMLPV2, self).__init__()
        
        layers = []
        current_input_size = input_size
        
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(current_input_size, hidden_size))
            layers.append(nn.ReLU())
            current_input_size = hidden_size
        
        layers.append(nn.Linear(current_input_size, output_size))
        layers.append(nn.ReLU())
        self.network = nn.Sequential(*layers)
    
    def forward(self, x1, x2):
        x1 = self.network(x1)
        x2 = self.network(x2)
        
        return torch.norm(x1 - x2, dim=1)

In [46]:
class CustomLoss(nn.Module):
    def __init__(self, margin):
        super(CustomLoss, self).__init__()
        self.margin = margin

    def forward(self, distance, target):
        loss = (distance**2)*target + (1-target)*(torch.max(self.margin - distance, torch.zeros_like(target))**2)
        return torch.mean(loss).squeeze()

In [47]:
SAVES_FOLDER = "saves/"
df = pd.read_csv(SAVES_FOLDER + "dataset.csv")

with open(SAVES_FOLDER + 'id2embedding.pkl', 'rb') as f:
    id2embedding = pickle.load(f)

In [48]:
features_cols = ["left_spec_id", "right_spec_id"]
target_col = "label"

multiplier = 100
dataset = CustomDataset(df, features_cols, target_col, id2embedding, multiplier, True)

train_size = int(0.75 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

input_size = 768
hidden_layers = [256, 128, 64]
#input_size = 1024
#hidden_layers = [384, 192, 96]

output_size = 48
model = SiameseMLPV2(input_size, hidden_layers, output_size)

margin = multiplier/5.0
criterion = CustomLoss(margin=margin)

optimizer = optim.SGD(model.parameters(), lr=0.00025)

In [49]:
NUM_EPOCHS = 100
stats = dict()
for epoch in range(NUM_EPOCHS):
    loss = trainLoop(model, optimizer, criterion, train_loader)
    print(f"Epoch {epoch + 1}, Loss: {loss}")
    pred_function = lambda x: x < margin
    testLoop(model, criterion, test_loader, pred_function, stats)

Epoch 1, Loss: 114.14025458971659
Test Loss: 85.9456, Test Accuracy: 0.4400
Precision: 0.3801, Recall: 0.9783, F1-score: 0.5475
TP: 271, FP: 442, TN: 81, FN: 6
Epoch 2, Loss: 74.73170240402222
Test Loss: 75.3530, Test Accuracy: 0.5713
Precision: 0.4448, Recall: 0.9603, F1-score: 0.6080
TP: 266, FP: 332, TN: 191, FN: 11
Epoch 3, Loss: 58.88523851076762
Test Loss: 65.5017, Test Accuracy: 0.6062
Precision: 0.4668, Recall: 0.9639, F1-score: 0.6290
TP: 267, FP: 305, TN: 218, FN: 10
Epoch 4, Loss: 50.86327011267344
Test Loss: 55.8002, Test Accuracy: 0.6312
Precision: 0.4840, Recall: 0.9856, F1-score: 0.6492
TP: 273, FP: 291, TN: 232, FN: 4
Epoch 5, Loss: 43.3371971420447
Test Loss: 49.2094, Test Accuracy: 0.7087
Precision: 0.5440, Recall: 0.9819, F1-score: 0.7001
TP: 272, FP: 228, TN: 295, FN: 5
Epoch 6, Loss: 38.57216187198957
Test Loss: 46.5826, Test Accuracy: 0.7025
Precision: 0.5386, Recall: 0.9819, F1-score: 0.6957
TP: 272, FP: 233, TN: 290, FN: 5
Epoch 7, Loss: 33.93764830775559
Test L

In [50]:
save_stats("MLP_V2_bert_base", stats)